In [1]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=4
# handle_num=11


exampleName="cow"
poses_folder="poses-2"
poses_ind=5
clip_k=4
handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21



weights = {'W_spatial': 0.0}

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cow-all/poses-2/cow-1.obj', './models/cow-all/poses-2/cow-2.obj', './models/cow-all/poses-2/cow-3.obj', './models/cow-all/poses-2/cow-4.obj', './models/cow-all/poses-2/cow-5.obj', './models/cow-all/poses-2/cow-6.obj', './models/cow-all/poses-2/cow-7.obj']
(11666, 28)
(11666, 21)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])

print (grad_zero_indices[:len(endmembers)*3])

print (mixing_weights_clip.ravel()[grad_zero_indices].sum())

(15, 84)
(11666, 15)
[[0.20570602 0.00042219 0.02655054 0.15167153 0.00018981 0.00004767
  0.00030539 0.00002423 0.02001831 0.00000105 0.00102998 0.14445015
  0.00045419 0.44910447 0.00002447]
 [0.21555833 0.0004373  0.02679411 0.16466939 0.00016431 0.00004939
  0.00029969 0.00002367 0.01424946 0.00000119 0.00101018 0.15187666
  0.00048589 0.42435294 0.00002749]
 [0.24024097 0.00040106 0.02997077 0.15734726 0.00013956 0.00004676
  0.00030475 0.00002278 0.01438586 0.00000122 0.00103343 0.11445163
  0.00051674 0.44110425 0.00003297]]
[[0.20570602 0.         0.         0.15167153 0.         0.
  0.         0.         0.         0.         0.         0.14445015
  0.         0.44910447 0.        ]
 [0.21555833 0.         0.         0.16466939 0.         0.
  0.         0.         0.         0.         0.         0.15187666
  0.         0.42435294 0.        ]
 [0.24024097 0.         0.         0.15734726 0.         0.
  0.         0.         0.         0.         0.         0.11445163
  0.  

In [5]:
scale=find_scale(vertices1)
print (scale)

708.6870049193498


In [6]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.07533573276941447


In [7]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

37144.77739542564
(174990, 1)
(11666, 28)
(11666, 21)
11666 7 15
W_spatial 0.0
     pcost       dcost       gap    pres   dres
 0: -4.9599e+07 -5.0121e+07  5e+06  4e+00  6e-13
 1: -4.9599e+07 -5.0067e+07  3e+06  3e+00  6e-13
 2: -4.9580e+07 -4.9899e+07  3e+06  2e+00  6e-13
 3: -4.9472e+07 -4.9253e+07  3e+06  2e+00  6e-13
 4: -4.9385e+07 -4.8820e+07  3e+06  2e+00  7e-13
 5: -4.8948e+07 -4.7838e+07  3e+06  1e+00  6e-13
 6: -4.8480e+07 -4.7108e+07  3e+06  1e+00  6e-13
 7: -4.8309e+07 -4.6783e+07  3e+06  1e+00  6e-13
 8: -4.7723e+07 -4.5944e+07  3e+06  7e-01  6e-13
 9: -4.7219e+07 -4.5063e+07  3e+06  6e-01  5e-13
10: -4.6767e+07 -4.4185e+07  3e+06  5e-01  5e-13
11: -4.6435e+07 -4.3275e+07  3e+06  5e-01  5e-13
12: -4.6293e+07 -4.2349e+07  3e+06  5e-01  6e-13
13: -4.6140e+07 -4.0665e+07  4e+06  4e-01  6e-13
14: -4.5640e+07 -3.8236e+07  4e+06  4e-01  6e-13
15: -4.4743e+07 -3.5278e+07  5e+06  4e-01  6e-13
16: -4.3478e+07 -3.1990e+07  6e+06  3e-01  6e-13
17: -4.3210e+07 -2.8951e+07  7e+06  3e-0

In [8]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print (res.ravel()[grad_zero_indices].sum()) ### should be zero
print (res.ravel()[grad_zero_indices].max()) ### should be zero
print (res.sum())

[[ 0.34404205  0.1080483  -0.01678851  0.34759121 -0.01501589 -0.01647529
  -0.01515092 -0.00974305  0.35430319 -0.00315285 -0.01536651 -0.01699708
  -0.01614025 -0.01710094 -0.01205345]
 [ 0.16045388 -0.0159742   0.1960362   0.15113497  0.04682882 -0.01151912
  -0.01558226 -0.01152185  0.18173544 -0.00402929 -0.01581303  0.15609359
   0.04478771  0.14975181 -0.01238268]
 [-0.01709864  0.05514794 -0.01638057 -0.01646635 -0.01210484  0.03631999
  -0.0153973  -0.01397726  0.53034547 -0.0029637  -0.01486401  0.53272025
  -0.01587188 -0.01708701 -0.0123221 ]]
4256.591863514781
1.017104944512035
11666.000000000045


In [9]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

3.9379638865665833e-25
-0.017420422698964173 1.0171398912454588


In [10]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

83964547292262.84


In [11]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [12]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cow-all/cow.DMAT']
(11666, 15)
1.5653958587979453e-28


In [13]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cow-all/poses-2/cow-1.Tmat', './models/cow-all/poses-2/cow-2.Tmat', './models/cow-all/poses-2/cow-3.Tmat', './models/cow-all/poses-2/cow-4.Tmat', './models/cow-all/poses-2/cow-5.Tmat', './models/cow-all/poses-2/cow-6.Tmat', './models/cow-all/poses-2/cow-7.Tmat']
(15, 84)
(7, 15, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.998956    0.0322376   0.0323555  -2.24837   ]
 [-0.0456047   0.664913    0.745527   27.1422    ]
 [ 0.00252045 -0.746225    0.665689   53.4901    ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.99999923 -0.00000004  0.00000002]
 [-0.00000004  1.00000031 -0.00000012]
 [ 0.00000002 -0.00000012  0.99999923]]


In [14]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]
recovered_handle_position=mixing_weights.T.dot(Vertices)/mixing_weights.T.sum(axis=-1)[...,np.newaxis]
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]
postprocessed_handle_position=res.T.dot(Vertices)/res.T.sum(axis=-1)[...,np.newaxis]

%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


Using matplotlib backend: MacOSX
